# BEE 4750: Final Project

Members: Jonathan Marcuse (jrm564), Geovany Rodriguez (ga283), Oliver Sandiford (ots22)

In [115]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `~/Desktop/Final Projects/4750_Project`


In [116]:
#initiating several packages that may be useful later
Pkg.add("Plots")
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Dates")
Pkg.add("Distributions")
using Plots
using CSV
using DataFrames
using Dates
using Distributions

   Resolving package versions...


  No Changes to `~/Desktop/Final Projects/4750_Project/Project.toml`
  No Changes to `~/Desktop/Final Projects/4750_Project/Manifest.toml`


   Resolving package versions...


  No Changes to `~/Desktop/Final Projects/4750_Project/Project.toml`
  No Changes to `~/Desktop/Final Projects/4750_Project/Manifest.toml`


   Resolving package versions...


  No Changes to `~/Desktop/Final Projects/4750_Project/Project.toml`
  No Changes to `~/Desktop/Final Projects/4750_Project/Manifest.toml`


   Resolving package versions...


  No Changes to `~/Desktop/Final Projects/4750_Project/Project.toml`
  No Changes to `~/Desktop/Final Projects/4750_Project/Manifest.toml`


   Resolving package versions...


  No Changes to `~/Desktop/Final Projects/4750_Project/Project.toml`
  No Changes to `~/Desktop/Final Projects/4750_Project/Manifest.toml`


#### Implementing the first component of the model - Determining how many aircrafts can land per hour

In [117]:
#Initialize flexible variables
aircraft_landing_speeds = [130 137.5 145 152.5 160]; #landing speeds of each type of aircraft in knots (interpolated)
proportions = [0.101333333 0.248 0.242666667 0.370666667 0.037333333]; #proportion of each type of aircraft, corresponding to the number of ones listed above
delta = 3; #minimum spacing between aircrafts in nautical miles (safety parameter) - taken from CEE 4640 safety parameter
gamma = [(10000/6076.11549) (6800/6076.11549) (11000/6076.11549)]; #length of the runway in nautical miles for each runway
#converted length in ft to nautical miles with 6,076.11549 ft per nautical mile
#The first 2 runways are for landing only and the third is for takeoff only

1×3 Matrix{Float64}:
 1.64579  1.11914  1.81037

In [118]:
for k in 1:2
    l = length(aircraft_landing_speeds);
    Time_Space = zeros(l,l);
    Time_Probabilities = zeros(l,l);
    for i in 1:l
        for j in 1:l
            Time_Space[i,j] = aircraft_landing_speeds[i] > aircraft_landing_speeds[j] ? ((delta/aircraft_landing_speeds[i])*3600) : (((delta/aircraft_landing_speeds[i])+(gamma[k]/aircraft_landing_speeds[i])-(gamma[k]/aircraft_landing_speeds[j]))*3600)
            Time_Probabilities[i,j] = proportions[i]*proportions[j]*Time_Space[i,j];
        end
    end
    avg_time = sum(Time_Probabilities); #total average seconds between aircraft landings
    aircrafts_per_hour = floor((1/avg_time)*3600); #floor functin used because you cant have a fraction of an aircraft land
    println("The average amount of time in between aircraft landings each hour on runway $k is $avg_time seconds")
    println("The total number of aircrafts that can land per hour on average on runway $k is $aircrafts_per_hour aircrafts/hour")
    println(" ")
end
k=3
l = length(aircraft_landing_speeds);
Time_Space = zeros(l,l);
Time_Probabilities = zeros(l,l);
for i in 1:l
    for j in 1:l
        Time_Space[i,j] = aircraft_landing_speeds[i] > aircraft_landing_speeds[j] ? ((delta/aircraft_landing_speeds[i])*3600) : (((delta/aircraft_landing_speeds[i])+(gamma[k]/aircraft_landing_speeds[i])-(gamma[k]/aircraft_landing_speeds[j]))*3600)
        Time_Probabilities[i,j] = proportions[i]*proportions[j]*Time_Space[i,j];
    end
end
avg_time = sum(Time_Probabilities); #total average seconds between aircraft landings
aircrafts_per_hour = floor((1/avg_time)*3600); #floor functin used because you cant have a fraction of an aircraft land
println("The average amount of time in between aircraft takeoffs each hour on runway $k is $avg_time seconds")
println("The total number of aircrafts that can take off per hour on average on runway $k is $aircrafts_per_hour aircrafts/hour")


The average amount of time in between aircraft landings each hour on runway 1 is 76.02652327219118 seconds
The total number of aircrafts that can land per hour on average on runway 1 is 47.0 aircrafts/hour
 
The average amount of time in between aircraft landings each hour on runway 2 is 75.61570264937018 seconds
The total number of aircrafts that can land per hour on average on runway 2 is 47.0 aircrafts/hour
 
The average amount of time in between aircraft takeoffs each hour on runway 3 is 76.15490471682274 seconds
The total number of aircrafts that can take off per hour on average on runway 3 is 47.0 aircrafts/hour


The first matrix above is the needed spacing between aircrafts landing in seconds, where the first row and first column are the speed of the first aircraft listed in the aircraft_landing_speeds vector. The second matrix above is the proportion of each type of aircraft multiplied by each other and then by the time spacing to form the time probability matrix that will later be used to find the total average time between aircrafts by summing all of its elements.

---

- historical runway usage --> flight demand data
- weather conditions --> what weather conditions cause flight to be cancelled
- Air traffic volume & air wake concerns--> model already built out
- passenger load and unload time

Average number of days per month with wind speeds above 38mph:
January had 1.7 days above 38mph, 1.7 feb, 1.6 days in march, 0.2 april, 0 may, 0 june, 0 july, 0 august, 0 sept, 0.1 oct, 0.4 nov, 1.9 dec

---

### Create A Distribution of All Planes

In [119]:
#value is the speed of the selected aircraft as this is what is relevance for wake calculations
aircraft_landing_speeds = [130, 137.5, 145, 152.5, 160]; 
proportions = [0.101333333, 0.248, 0.242666667, 0.370666667, 0.037333333]; #proportions of each
plane_dist = Categorical(proportions);
#Creating the distribution to select planes from 
#plane_distribution = Weighted(aircraft_landing_speeds, proportions);


### for iteration codeblock
#selected_plane = rand(plane_distribution);

Categorical{Float64, Vector{Float64}}(
support: Base.OneTo(5)
p: [0.101333333, 0.248, 0.242666667, 0.370666667, 0.037333333]
)


### Create function to find separation time between two types of aircrafts

In [120]:
function wake_time(plane1_speed,plane2_speed,min_spacing,runway_length)
    Time_Space = plane1_speed > plane2_speed ? 
    ((min_spacing/plane1_speed)*3600) : 
    (((min_spacing/plane1_speed)+(runway_length/plane1_speed)
    -(runway_length/plane2_speed))*3600);
    return Time_Space
end

wake_time (generic function with 1 method)

---

### Monte Carlo Iteration of EWR Air Traffic Model

In [159]:
days = zeros(5000);
#Monte Carlo loop
for j in 1:5000

    tot_planes_per_week = 8621; #planes per week
    min_spacing = 3; #nautical miles (safety parameter for distance between aircrafts in air)
    runway1_length = (10000/6076.11549); #nautical miles for runway 1
    runway2_length = (11000/6076.11549); #nautical miles for runway 2
    t = zeros(Float64, 2); #initial time at beginning of iteration where position 2 a tracker for time related to arrivals and position 1 is related to departures
    i=1; #initialize iteration stepper

    full_gates = 72; #60% of 121 gates full initially and this can never go above 121
    plane = zeros(tot_planes_per_week); #initialize
    selected_plane = rand(plane_dist);
    plane[i] = aircraft_landing_speeds[selected_plane] #select the first for comparison to the next in the for loop

    while full_gates <=121 && i<=tot_planes_per_week-1 #loop fails if gate capacity fills and terminates if all planes land and depart
        i = i+1 #iterate plane number of the week
        selected_plane = rand(plane_dist); #randomly select an index 1 through 5 based on weighted distribution
        plane[i] = aircraft_landing_speeds[selected_plane]; #use the randomly chosen index to get the associated plane landing/takeoff speed

        landing_wake_time_separation = (wake_time(plane[i],plane[i-1],min_spacing,runway1_length))/2; #divided by 2 to assume two runways used
        t[2] = t[2]+landing_wake_time_separation; #time step of wake separation times
        full_gates = full_gates+1; #one new plane at the gate

        #time delays after landing
        deplane_and_boarding = 90*60+120; #90 minutes per plane to board and deplane and 2 min for taxi
        takeoff_wake_time_separation = wake_time(plane[i-1],plane[i],min_spacing,runway1_length); #reversed planes and not divided by 2 to make sure every landing plane also takes off, but at half the separation time
        time_for_departure = max(takeoff_wake_time_separation,deplane_and_boarding/full_gates); #amount of time for each gate to be ready to fly another plane
        
        if t[2]-t[1]>=time_for_departure #situation where a departure has occured since the last arrival
            departures = floor((t[2]-t[1])/time_for_departure); #number of departures that could've occured in this delta t
            full_gates = full_gates-departures; #subtract number of full gates
            t[1] = t[1]+time_for_departure*departures; #additional time iteration for departures
        elseif t[2]-t[1]<time_for_departure && full_gates==121 #situation where no departure since last arrival and gates are all full
            t[2] = t[1]+time_for_departure; #time step increase to have arrivals delayed until gate space is open
            full_gates = full_gates-1;
        end
    end

    days[j] = t[2]/86400;
end

println("It will take $(round(sum(days)/length(days),digits=4)) days for all $i planes to land and departure
compared to the 7 days for 8621 planes to land under normal opertations")

It will take 4.5784 days for all 8621 planes to land and departure
compared to the 7 days for 8621 planes to land under normal opertations


Assumptions made in the above:
- starting gate capacity will be 72 out of 121, but for the majority of the rest of the week it will run at full capacity (can constrain lower if needed)
- total time between takeoffs will be the wake time separation between aircrafts OR the waiting time for deplane, boarding and taxi divided by the total number of gates which gives you the total time in between filled gates with planes ready to depart: whichever of these 2 is greatest will be used
- total time between landings will be the wake separation equation because it also includes length of the runway parameter to ensure it is cleared before the next landing
- Not really an assumption but an added explanation to the above code: The takeoffs and landings must be coordinated such that the total number of grounded planes filling gates is not exceeded which is why we have the if statements for additional time steps


Still need to:
- Add delays from weather
- Get something to be graphed
- Total people transported in alloted timeframe
- Associated Emissions? Needs some environmental aspect

---

### Load in Wind Speed Data
This has the wind speeds at Newark for every 3 minutes of the month of August in 2023. This will be used to tell us how much to delay flights by given the wind speed. Based on our background research, if the wind speed is between 30mph and 40pmh, the aircraft will be delayed by 30 minutes after the latest instance of winds in that speed. If the wind speed is greater than 40mph, the aircraft will be delayed by 1 hour from the latest instance of that wind speed. If the wind speed is under 30mph it is clear for takeoff.

In [122]:
# Load the CSV file into a DataFrame
Wind_DataFrame = DataFrame(CSV.File("Wind_Data.csv"));

# Extract wind speed and date columns
wind_speed = Wind_DataFrame."Wind Speed (mph)";
date_string = Wind_DataFrame."Date";

# Parse the date string with the correct format
date_value = Dates.DateTime.(date_string, "mm/dd/yyyy HH:MM");